In [1]:
import pandas as pd
import numpy as np
import json
import ast
import csv
import seaborn as sns
import matplotlib.pyplot as plt
from tabulate import tabulate

In [2]:
#Helper functions
def parse_json_safe(x):
    if pd.isna(x) or x == '':
        return []
    try:
        return json.loads(x) if isinstance(x, str) else x
    except Exception:
        try:
            return ast.literal_eval(x)
        except Exception:
            return []

def pretty_print(df, title=None, n=5):
    if title:
        print(f"\n===== {title} =====")
    print(tabulate(df.head(n), headers="keys", tablefmt="fancy_grid"))
    if len(df) > n:
        print(f"... ({len(df) - n} more rows)")


In [3]:
bad_movie_lines = []
bad_credit_lines = []
bad_keyword_lines = []
bad_links_lines = []
bad_ratings_lines = []

def log_bad_movie(line):
    bad_movie_lines.append(line)
    return None

def log_bad_credit(line):
    bad_credit_lines.append(line)
    return None

def log_bad_keyword(line):
    bad_keyword_lines.append(line)
    return None

def log_bad_links(line):
    bad_links_lines.append(line)
    return None

def log_bad_ratings(line):
    bad_ratings_lines.append(line)

movies = pd.read_csv(
    "../movies/movies_metadata.csv",
    engine="python",
    sep=",",
    quotechar='"',
    on_bad_lines=lambda line: log_bad_movie(line)
)

movie_credits = pd.read_csv(
    "../movies/credits.csv",
    engine="python",
    sep=",",
    quotechar='"',
    on_bad_lines=lambda line: log_bad_credit(line)
)

keywords = pd.read_csv(
    "../movies/keywords.csv",
    engine="python",
    sep=",",
    quotechar='"',
    on_bad_lines=lambda line: log_bad_keyword(line)
)

links = pd.read_csv(
    "../movies/links.csv",
    engine="python",
    sep=",",
    quotechar='"',
    on_bad_lines=lambda line: log_bad_movie(line)
)

ratings = pd.read_csv(
    "../movies/ratings.csv",
    engine="python",
    sep=",",
    quotechar='"',
    on_bad_lines=lambda line: log_bad_movie(line)
)

print("Datasets loaded.")
print(f"movies: {movies.shape}")
print(f"credits: {movie_credits.shape}")
print(f"keywords: {keywords.shape}")
print(f"links: {links.shape}")
print(f"ratings: {ratings.shape}")

print(f"movies skipped lines: {len(bad_movie_lines)}")
print(f"credits skipped lines: {len(bad_credit_lines)}")
print(f"keywords skipped lines: {len(bad_keyword_lines)}")
print(f"links skipped lines: {len(bad_links_lines)}")
print(f"ratings skipped lines: {len(bad_ratings_lines)}")

if bad_movie_lines:
    print("example bad movie line:")
    print(bad_movie_lines[0][:200])
else:
    print("No bad movie lines found.")
if bad_credit_lines:
    print("example bad credit line:")
    print(bad_credit_lines[0][:200])
else:
    print("No bad credit lines found.")
if bad_keyword_lines:
    print("example bad keyword line:")
    print(bad_keyword_lines[0][:200])
else:
    print("No bad keyword lines found.")
if bad_links_lines:
    print("example bad links:")
    print(bad_links_lines[0][:200])
else:
    print("No bad links lines found.")
if bad_ratings_lines:
    print("example bad ratings:")
    print(bad_ratings_lines[0][:200])
else:
    print("No bad ratings lines found.")


Datasets loaded.
movies: (45466, 24)
credits: (45476, 3)
keywords: (46419, 2)
links: (45843, 3)
ratings: (26024289, 4)
movies skipped lines: 0
credits skipped lines: 0
keywords skipped lines: 0
links skipped lines: 0
ratings skipped lines: 0
No bad movie lines found.
No bad credit lines found.
No bad keyword lines found.
No bad links lines found.
No bad ratings lines found.


In [4]:
import pandas as pd

def data_overview(df, name):
    print(f"\n===== Overview of {name} =====")
    summary = pd.DataFrame({
        "dtype": df.dtypes,
        "unique_values": df.nunique(),
        "missing_count": df.isnull().sum(),
        "missing_percent": (df.isnull().sum() / len(df) * 100).round(2)
    })

    summary = summary.sort_values("missing_count", ascending=False)
    print(summary)

    for col in df.columns:
        unique_count = df[col].nunique(dropna=True)
        if unique_count < 10:
            print("\n--- Columns with < 10 Unique Values ---")
            unique_vals = df[col].dropna().unique()
            print(f"\n{col} ({unique_count} unique values):")
            print(unique_vals)


data_overview(movies, "Movies Metadata")
data_overview(movie_credits, "Credits")
data_overview(keywords, "Keywords")
data_overview(links, "Links Small")
data_overview(ratings, "Ratings Small")


===== Overview of Movies Metadata =====
                         dtype  unique_values  missing_count  missing_percent
belongs_to_collection   object           1698          40972            90.12
homepage                object           7673          37684            82.88
tagline                 object          20283          25054            55.10
overview                object          44307            954             2.10
poster_path             object          45024            386             0.85
runtime                float64            353            263             0.58
status                  object              6             87             0.19
release_date            object          17336             87             0.19
imdb_id                 object          45417             17             0.04
original_language       object             92             11             0.02
vote_average           float64             92              6             0.01
vote_count             

In [5]:
def find_duplicates(df, keys, name=None, show_examples=True):
    if name:
        print(f"\n--- Checking Duplicates in {name} ---")

    missing_keys = [k for k in keys if k not in df.columns]
    if missing_keys:
        print(f"Missing key columns: {missing_keys}")
        return pd.DataFrame()

    duplicate_mask = df.duplicated(subset=keys, keep=False)
    key_dupes_df = df.loc[duplicate_mask].sort_values(by=keys)

    duplicate_key_groups = (
        key_dupes_df[keys]
        .drop_duplicates()
        .shape[0]
        if not key_dupes_df.empty
        else 0
    )

    full_duplicates = df.duplicated(keep=False)
    full_dupes_df = df.loc[full_duplicates]
    full_dupes_count = full_dupes_df.drop_duplicates().shape[0] if not full_dupes_df.empty else 0

    partial_dupes_df = key_dupes_df.merge(
        full_dupes_df.drop_duplicates(),
        how="left",
        indicator=True
    ).query('_merge == "left_only"').drop(columns="_merge")

    if not partial_dupes_df.empty:
        diff_info = []
        for key_values, group in partial_dupes_df.groupby(keys):
            # find columns with more than one unique non-null value in this group
            differing_cols = [
                col for col in group.columns
                if col not in keys and group[col].nunique(dropna=False) > 1
            ]
            group = group.copy()
            group["diff_columns"] = ", ".join(differing_cols) if differing_cols else np.nan
            diff_info.append(group)
        partial_dupes_df = pd.concat(diff_info, ignore_index=True)

    print(f"Duplicate groups based on keys {keys}: {duplicate_key_groups}")
    print(f"Fully duplicate rows (identical across all columns): {full_dupes_count}")

    if not partial_dupes_df.empty and show_examples:
        print("\nExample duplicate rows based on keys:")
        print(partial_dupes_df.head(10))

    return key_dupes_df


find_duplicates(movies, ['id'], "Movies Metadata")
find_duplicates(movie_credits, ['id'], "Credits")
find_duplicates(keywords, ['id'], "Keywords")
find_duplicates(links, ['movieId'], "Links")
find_duplicates(ratings, ['movieId', 'userId'], "Ratings")


--- Checking Duplicates in Movies Metadata ---
Duplicate groups based on keys ['id']: 29
Fully duplicate rows (identical across all columns): 16

Example duplicate rows based on keys:
   adult                              belongs_to_collection    budget  \
0  False  {'id': 34055, 'name': 'Pokémon Collection', 'p...  16000000   
1  False  {'id': 34055, 'name': 'Pokémon Collection', 'p...  16000000   
2  False                                                NaN         0   
3  False                                                NaN         0   
4  False                                                NaN   3512454   
5  False                                                NaN   3512454   
6  False  {'id': 34055, 'name': 'Pokémon Collection', 'p...         0   
7  False  {'id': 34055, 'name': 'Pokémon Collection', 'p...         0   
8  False                                                NaN      2500   
9  False                                                NaN      2500   

           

,userId,movieId,rating,timestamp


In [12]:
# --- Movies ---
if "id" in movies.columns:
    movies["id"] = pd.to_numeric(movies["id"], errors="coerce").astype("Int64")
    invalid = movies["id"].isna().sum()
    print(f"movies: converted 'id' to numeric ({invalid} invalid IDs)")

# --- Credits ---
if "id" in movie_credits.columns and "movie_id" not in movie_credits.columns:
    movie_credits.rename(columns={"id": "movie_id"}, inplace=True)
movie_credits["movie_id"] = pd.to_numeric(movie_credits["movie_id"], errors="coerce").astype("Int64")
invalid = movie_credits["movie_id"].isna().sum()
print(f"credits: normalized 'movie_id' ({invalid} invalid IDs)")

# --- Keywords ---
if "id" in keywords.columns and "movie_id" not in keywords.columns:
    keywords.rename(columns={"id": "movie_id"}, inplace=True)
keywords["movie_id"] = pd.to_numeric(keywords["movie_id"], errors="coerce").astype("Int64")
invalid = keywords["movie_id"].isna().sum()
print(f"keywords: normalized 'movie_id' ({invalid} invalid IDs)")

# --- Links ---
# Normalize TMDb ID
if "tmdbId" in links.columns:
    links.rename(columns={"tmdbId": "tmdb_id"}, inplace=True)
    links["tmdb_id"] = pd.to_numeric(links["tmdb_id"], errors="coerce").astype("Int64")
    invalid = links["tmdb_id"].isna().sum()
    print(f"links: normalized 'tmdb_id' ({invalid} invalid IDs)")

# Normalize MovieLens ID
if "movieId" in links.columns:
    links.rename(columns={"movieId": "movie_lens_id"}, inplace=True)
links["movie_lens_id"] = pd.to_numeric(links["movie_lens_id"], errors="coerce").astype("Int64")

# Normalize IMDb ID (optional, if you want)
if "imdbId" in links.columns:
    links.rename(columns={"imdbId": "imdb_id"}, inplace=True)
    links["imdb_id"] = pd.to_numeric(links["imdb_id"], errors="coerce").astype("Int64")


# --- Ratings ---
if "movieId" in ratings.columns:
    ratings.rename(columns={"movieId": "movie_lens_id"}, inplace=True)
ratings["movie_lens_id"] = pd.to_numeric(ratings["movie_lens_id"], errors="coerce").astype("Int64")

movies: converted 'id' to numeric (3 invalid IDs)
credits: normalized 'movie_id' (0 invalid IDs)
keywords: normalized 'movie_id' (0 invalid IDs)
links: normalized 'tmdb_id' (219 invalid IDs)


In [13]:
def count_invalid_references(df, id_col, name, valid_ids):
    if id_col not in df.columns:
        print(f"{name}: no '{id_col}' column, skipped validation")
        return
    total = len(df)
    invalid = (~df[id_col].isin(valid_ids)).sum()
    print(f"{name}: {invalid} invalid '{id_col}' references out of {total} rows")


# --- Prepare sets of valid IDs ---
valid_tmdb_ids = set(movies["id"].dropna())
valid_link_ids = set(links["movie_lens_id"].dropna())

# --- Validate ---
# Credits → Movies
count_invalid_references(movie_credits, "movie_id", "credits", valid_tmdb_ids)

# Keywords → Movies
count_invalid_references(keywords, "movie_id", "keywords", valid_tmdb_ids)

# Links → Movies
count_invalid_references(links, "tmdb_id", "links", valid_tmdb_ids)

# Ratings → Links
count_invalid_references(ratings, "movie_lens_id", "ratings", valid_link_ids)


credits: 0 invalid 'movie_id' references out of 45476 rows
keywords: 0 invalid 'movie_id' references out of 46419 rows
links: 380 invalid 'tmdb_id' references out of 45843 rows
ratings: 0 invalid 'movie_lens_id' references out of 26024289 rows
